<a href="https://colab.research.google.com/github/bdnguye2-a11y/S2023_EEE587_Team2/blob/working_branch/New_linear_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install control


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.8/432.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
from tqdm import *
import control as ctl

sp.init_printing(use_unicode=True)


print('Initializing Variables...')
c = lambda x: sp.cos(x)
s = lambda x: sp.sin(x)
syms = []

syms.append(dynamicsymbols('x'))
syms.append(sp.diff(syms[-1]))
syms.append(dynamicsymbols('y'))
syms.append(sp.diff(syms[-1]))
syms.append(dynamicsymbols('z'))
syms.append(sp.diff(syms[-1]))


syms.append(dynamicsymbols('alpha'))
syms.append(sp.diff(syms[-1]))
syms.append(dynamicsymbols('beta'))
syms.append(sp.diff(syms[-1]))




t = sp.symbols('t')

x = syms[0]
xdot = syms[1]
y = syms[2]
ydot = syms[3]
z = syms[4]
zdot = syms[5]


alpha = syms[6]
alphadot = syms[7]
beta = syms[8]
betadot = syms[9]

psi=dynamicsymbols('psi')
phi=dynamicsymbols('phi')
theta=dynamicsymbols('theta')
q = sp.Matrix([x,y,z,alpha,beta])

m = sp.symbols('m')
l = sp.symbols('l')
M = sp.symbols('M')
g = sp.symbols('g')

I_psi = sp.symbols('I_psi')
I_theta = sp.symbols('I_theta')
I_phi = sp.symbols('I_phi')

I_p = sp.symbols('I_p')


zeta = sp.Matrix([x,y,z])
r = sp.Matrix([s(alpha)*c(beta),s(alpha)*s(beta),-c(alpha)])
zetap = zeta+l*r
eta = sp.Matrix([psi,theta,phi])

J = sp.zeros(3,3)

s_th = s(theta)
c_th = c(theta)
c_ph = c(phi)
s_ph = s(phi)

zetadot = sp.diff(zeta,t)
etadot = sp.diff(eta,t)

zetapdot = sp.diff(zetap,t)

m11 = M+m
m22 = m11
m33 = m11
m14 = m*l*c(alpha)*c(beta)
m15 = -m*l*s(alpha)*s(beta)
m25 = m*l*c(beta) 
m34 = -m*l*s(alpha)*c(beta)
m35 = -m*l*c(alpha)*s(beta)
m41 = m14
m43 = m34
m51 = m15
m52 = m25
m53 = m35
m44 = m*(l**2)*(c(beta)**2)
m45 = 0
m55 = m*l**2


M_q = sp.Matrix([[m11,0,0,m14, m15],
                 [0,m22,0, 0, m25],
                 [0,0,m33,m34, m35],
                 [m41,0,m43,m44,m45],
                 [m51,m52,0,0,m55]])

c14 = -(c(alpha)*s(beta)*betadot- s(alpha)*c(beta)*alphadot)
c15 = -(c(alpha)*s(beta)*alphadot - s(alpha)*c(beta)*betadot)
c24 = 0
c25 = -betadot*s(beta)
c34 = (-c(alpha)*c(beta)*alphadot + s(alpha)*s(beta)*betadot)
c35 = (s(alpha)*s(beta)*alphadot - c(alpha)*c(beta)*betadot)
c44 = -l*betadot*c(beta)*s(beta)
c45 = -l*alphadot*c(beta)*s(beta)
c54 = l*alphadot*s(beta)*c(beta)
c55 = 0

C_q = sp.Matrix([[0,0,0,c14, c15],
                 [0,0,0,c24, c25],
                 [0,0,0,c34, c35],
                 [0,0,0,c44, c45],
                 [0,0,0,c54, c55]])
C_q = C_q*m*l
G_q = sp.Matrix([0,0,-(M+m)*g, m*l*g*s(alpha)*c(beta), m*l*g*c(alpha)*s(beta)])


inpts= sp.symbols('f_all')
U = sp.zeros(5,1)

U[0,0] = s(alpha)*s(psi)+c(phi)*c(psi)*s(theta)
U[1,0] = c(phi)*s(theta)*s(psi)-c(psi)*s(phi)
U[2,0] = c(theta)*c(phi)
U[3,0] = 0 
U[4,0] = 0


qdot = sp.diff(q,t)
qddot = sp.diff(qdot,t)

U=U*inpts
# u = sp.Matrix([inpts[0],inpts[1],inpts[2],inpts[3]])



Initializing Variables...


In [ ]:
X_dot = -M_q**-1*((C_q*qdot)+G_q-U)

In [ ]:
print(X_dot.shape)
q
qdot

(5, 1)


⎡d       ⎤
⎢──(x(t))⎥
⎢dt      ⎥
⎢        ⎥
⎢d       ⎥
⎢──(y(t))⎥
⎢dt      ⎥
⎢        ⎥
⎢d       ⎥
⎢──(z(t))⎥
⎢dt      ⎥
⎢        ⎥
⎢d       ⎥
⎢──(α(t))⎥
⎢dt      ⎥
⎢        ⎥
⎢d       ⎥
⎢──(β(t))⎥
⎣dt      ⎦

In [ ]:
cpy = sp.zeros(5,1)
for i in range(5):
    cpy[i] = X_dot[i]

params = [m,l,M,g]
vals = [1, 1, 7, 9.8]

for row in trange(5):
    for j in range(len(params)):
        cpy[row] = cpy[row].subs(params[j],vals[j])
        
# Linearization
#################################################
sbls = sp.symbols('x1:11')
for row in trange(5):
    for i in range(5):
        state_var = qdot[i]
        sub_sym = sbls[i+5]
        cpy[row] = cpy[row].subs(state_var,sub_sym)
    for i in range(5):
        state_var = q[i]
        sub_sym = sbls[i]
        cpy[row] = cpy[row].subs(state_var,sub_sym)

print(cpy.shape)
A_ = sp.zeros(10)
B_ = sp.zeros(10,1)


100%|██████████| 5/5 [00:00<00:00,  7.97it/s]

(5, 1)


In [ ]:
q

⎡x(t)⎤
⎢    ⎥
⎢y(t)⎥
⎢    ⎥
⎢z(t)⎥
⎢    ⎥
⎢α(t)⎥
⎢    ⎥
⎣β(t)⎦

In [ ]:
with tqdm(total = 5*20) as pbar:
    for row in range(5):
        A_[row,row+5] = 1
            
    for row in range(5):
        for column in range(10):
            A_[row+5,column] = sp.diff(cpy[row],sbls[column])
            pbar.update(1)
        for column in range(1):
            B_[row+5,column] = sp.diff(cpy[row],inpts)
            pbar.update(1)

print(B_.shape)

 55%|█████▌    | 55/100 [00:00<00:00, 115.30it/s]

(10, 1)


In [ ]:
########################################################
#x,y,z,psi,theta,phi,alpha,beta

def linearize(inQ,inU, a1, a2, a3):
    tempA = sp.zeros(10)
    tempB = sp.zeros(10,1)
    
    for row in range(10):
        for column in range(10):
            tempA[row,column] = A_[row,column]
    
    for row in range(10):
        for column in range(1):
            tempB[row,column] = B_[row,column]
    print(tempB.shape)
    for row in range(5,10):
        for j in range(10):
            for column in range(10):
                tempA[row,column] = tempA[row,column].subs(sbls[j],inQ[j])
                if column < 1:
                    tempB[row,column] = tempB[row,column].subs(sbls[j],inQ[j])
        for j in range(1):
            for column in range(10):
                tempA[row,column] = tempA[row,column].subs(inpts,inU)
                tempA[row,column] = tempA[row,column].subs(phi,a1)
                tempA[row,column] = tempA[row,column].subs(theta,a2)
                tempA[row,column] = tempA[row,column].subs(psi,a3)
                if column < 1:
                    tempB[row,column] = tempB[row,column].subs(inpts,inU)
                    tempB[row,column] = tempB[row,column].subs(phi,a1)
                    tempB[row,column] = tempB[row,column].subs(theta,a2)
                    tempB[row,column] = tempB[row,column].subs(psi,a3)
    # tempA.subs(psi, 0)
    # tempA.subs(theta, 0)
    # tempA.subs(phi, 0)
    print(tempA)
    print(tempB.shape)
    return np.array(tempA).astype(float),np.array(tempB).astype(float)

#x,y,z,psi,theta,phi,alpha,beta

eq = np.array([[1,0,1,0,np.pi/7, 0, 0, 0, 0, 0]])
initU = 1

A,B = linearize(eq[0,:],initU, 0.03, 0.01, 0.05)


(10, 1)
Matrix([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, -0.267162852316754, 0, 0, 0, 0, 0, 0], [0, 0, 0, -7.75560787799564e-5, 0.792504944244317, 0, 0, 0, 0, 0], [0, 0, 0, -0.000140915999893226, -1.03768554792838, 0, 0, 0, 0, 0], [0, 0, 0, 0.150555525155891, -0.000762272005136307, 0, 0, 0, 0, 0], [0, 0, 0, 0.000688646025787929, -9.31405734389033, 0, 0, 0, 0, 0]])
(10, 1)


In [ ]:
A

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.

In [ ]:
print(B)
print(B.shape)

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.00142612]
 [-0.00409814]
 [ 0.12513776]
 [-0.00158287]
 [ 0.00369229]]
(10, 1)


In [ ]:
C=np.zeros((5,10))
C[0,0]=1
C[1,1] =1
C[2,2] =1
C[3,3] =1
C[4,4] =1
sys = ctl.c2d(ctl.ss(A,B,C,0),.001)


In [ ]:
ctr=ctl.ctrb(A,B)
r=np.linalg.matrix_rank(ctr)

In [ ]:
print(r)

6


NameError: ignored

In [ ]:
n=10
Q = np.eye(n)

Q[0, 0] = 10.
Q[1, 1] = 10.
Q[2, 2] = 10.

Q[3, 3] = 5.
Q[4, 4] = 5.
Q[5, 5] = 5.
# Q[, 7] = 10.  # The first state variable is the one we care about.



print(Q)
R = np.diag([1.,])
R=1
D=0
print(B.shape)
sysg=ctl.ss(A,B,C,D)
sysd = sysg.sample(0.5, method='bilinear')
K, S, E = ctl.dlqr(sysd, Q, R)

[[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  5.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  5.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]
(10, 1)


ValueError: ignored